<a href="https://colab.research.google.com/github/tranvangaohd1994/STP_2021_2/blob/LinhDK/DeepLearning/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Thêm thư viện
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imutils import paths
from keras.applications.vgg16 import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/DeepLearning/STP-2019/flower-dataset.zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/DeepLearning/STP-2019/flower-dataset.zip
   creating: dataset/
   creating: dataset/Bluebell/
  inflating: dataset/Bluebell/image_0241.jpg  
  inflating: dataset/Bluebell/image_0242.jpg  
  inflating: dataset/Bluebell/image_0243.jpg  
  inflating: dataset/Bluebell/image_0244.jpg  
  inflating: dataset/Bluebell/image_0245.jpg  
  inflating: dataset/Bluebell/image_0246.jpg  
  inflating: dataset/Bluebell/image_0247.jpg  
  inflating: dataset/Bluebell/image_0248.jpg  
  inflating: dataset/Bluebell/image_0249.jpg  
  inflating: dataset/Bluebell/image_0250.jpg  
  inflating: dataset/Bluebell/image_0251.jpg  
  inflating: dataset/Bluebell/image_0252.jpg  
  inflating: dataset/Bluebell/image_0253.jpg  
  inflating: dataset/Bluebell/image_0254.jpg  
  inflating: dataset/Bluebell/image_0255.jpg  
  inflating: dataset/Bluebell/image_0256.jpg  
  inflating: dataset/Bluebell/image_0257.jpg  
  inflating: dataset/Bluebell/image_0258.jpg  
  inflating

In [ ]:
# Lấy các đường dẫn đến ảnh.
image_path = list(paths.list_images('dataset/'))
print(image_path)
# Đổi vị trí ngẫu nhiên các đường dẫn ảnh
random.shuffle(image_path)

['dataset/Tigerlily/image_0553.jpg', 'dataset/Tigerlily/image_0484.jpg', 'dataset/Tigerlily/image_0507.jpg', 'dataset/Tigerlily/image_0520.jpg', 'dataset/Tigerlily/image_0557.jpg', 'dataset/Tigerlily/image_0505.jpg', 'dataset/Tigerlily/image_0552.jpg', 'dataset/Tigerlily/image_0517.jpg', 'dataset/Tigerlily/image_0515.jpg', 'dataset/Tigerlily/image_0558.jpg', 'dataset/Tigerlily/image_0495.jpg', 'dataset/Tigerlily/image_0549.jpg', 'dataset/Tigerlily/image_0498.jpg', 'dataset/Tigerlily/image_0486.jpg', 'dataset/Tigerlily/image_0538.jpg', 'dataset/Tigerlily/image_0546.jpg', 'dataset/Tigerlily/image_0532.jpg', 'dataset/Tigerlily/image_0483.jpg', 'dataset/Tigerlily/image_0537.jpg', 'dataset/Tigerlily/image_0551.jpg', 'dataset/Tigerlily/image_0526.jpg', 'dataset/Tigerlily/image_0516.jpg', 'dataset/Tigerlily/image_0497.jpg', 'dataset/Tigerlily/image_0548.jpg', 'dataset/Tigerlily/image_0503.jpg', 'dataset/Tigerlily/image_0490.jpg', 'dataset/Tigerlily/image_0556.jpg', 'dataset/Tigerlily/image_05

In [ ]:
# Đường dẫn ảnh sẽ là dataset/tên_loài_hoa/tên_ảnh ví dụ dataset/Bluebell/image_0241.jpg nên p.split(os.path.sep)[-2] sẽ lấy ra được tên loài hoa
labels = [p.split(os.path.sep)[-2] for p in image_path]

# Chuyển tên các loài hoa thành số
le = LabelEncoder()
labels = le.fit_transform(labels)
print(labels)
# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels[1])

[ 0 11  7 ...  4  3  3]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]


In [ ]:
# Load ảnh và resize về đúng kích thước mà VGG 16 cần là (224,224)
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

In [ ]:
# Load model VGG 16 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
baseModel = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)
fcHead = Dropout(0.5)(fcHead)

# Output layer với softmax activation
fcHead = Dense(17, activation='softmax')(fcHead)

# Xây dựng model bằng việc nối ConvNet của VGG16 và fcHead
model = model = Model(inputs=baseModel.input, outputs=fcHead)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
# Chia traing set, test set tỉ lệ 80-20
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [ ]:
# augmentation cho training data
aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
                         zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# augementation cho test
aug_test= ImageDataGenerator(rescale=1./255)

In [ ]:
# freeze VGG model
for layer in baseModel.layers:
    layer.trainable = False
    
opt = RMSprop(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
numOfEpoch = 25
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
34/34 [==============================] - 56s 489ms/step - loss: 4.4668 - accuracy: 0.1700 - val_loss: 2.0412 - val_accuracy: 0.4141
Epoch 2/25
34/34 [==============================] - 17s 486ms/step - loss: 2.2239 - accuracy: 0.3088 - val_loss: 1.6296 - val_accuracy: 0.5859
Epoch 3/25
34/34 [==============================] - 17s 488ms/step - loss: 1.9422 - accuracy: 0.3768 - val_loss: 1.3519 - val_accuracy: 0.6016
Epoch 4/25
34/34 [==============================] - 17s 486ms/step - loss: 1.7265 - accuracy: 0.4430 - val_loss: 1.1287 - val_accuracy: 0.6836
Epoch 5/25
34/34 [==============================] - 17s 485ms/step - loss: 1.5519 - accuracy: 0.5156 - val_loss: 1.2186 - val_accuracy: 0.6055
Epoch 6/25
34/34 [==============================] - 17s 486ms/step - loss: 1.3414 - accuracy: 0.5533 - val_loss: 1.0365 - val_accuracy: 0.6406
Epoch 7/25
34/34 [==============================] - 17s 486ms/step - loss: 1.3439 - accuracy: 0.5662 - val_loss: 0.9467 - val_accuracy: 0.7031

In [ ]:
# unfreeze some last CNN layer:
for layer in baseModel.layers[15:]:
    layer.trainable = True

numOfEpoch = 35
opt = SGD(0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/35
34/34 [==============================] - 20s 505ms/step - loss: 0.5155 - accuracy: 0.8300 - val_loss: 0.4497 - val_accuracy: 0.8633
Epoch 2/35
34/34 [==============================] - 17s 505ms/step - loss: 0.4646 - accuracy: 0.8465 - val_loss: 0.4374 - val_accuracy: 0.8672
Epoch 3/35
34/34 [==============================] - 17s 508ms/step - loss: 0.4371 - accuracy: 0.8456 - val_loss: 0.4618 - val_accuracy: 0.8633
Epoch 4/35
34/34 [==============================] - 17s 503ms/step - loss: 0.3967 - accuracy: 0.8575 - val_loss: 0.3919 - val_accuracy: 0.8867
Epoch 5/35
34/34 [==============================] - 17s 505ms/step - loss: 0.4185 - accuracy: 0.8539 - val_loss: 0.4106 - val_accuracy: 0.8633
Epoch 6/35
34/34 [==============================] - 17s 505ms/step - loss: 0.3638 - accuracy: 0.8805 - val_loss: 0.4134 - val_accuracy: 0.8750
Epoch 7/35
34/34 [==============================] - 17s 508ms/step - loss: 0.3708 - accuracy: 0.8741 - val_loss: 0.4395 - val_accuracy: 0.8633